 creating using 1) the bboxes deleted by gunnar 2) some random patch on the fish

In [ ]:
import glob
import json
import numpy as np
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os

In [ ]:
cogito = glob.glob("/root/data/lice-data/crops/blom-kjeppevikholmen/2/*/*_cogito.json")
print('Number of cogito annotations: {}'.format(len(cogito)))
all_ann = glob.glob("/root/data/lice-data/crops/blom-kjeppevikholmen/2/*/*.json")
print('Number of gunnar annotations: {}'.format(len(all_ann) - len(cogito)))

In [ ]:
def find_difference(cog, gun):
    diff = []
    for l0 in cog:
        pos0 = l0['position']
        l0left = pos0['left']
        l0top = pos0['top']
        match = False
        for l1 in gun:
            pos1 = l1['position']
            l1left = pos1['left']
            l1top = pos1['top']
            if l1left == l0left and l1top == l0top:
                match = True
        if not match:
            diff.append(l0)
            
    return diff

In [ ]:
bad = []
for cogann in tqdm(cogito):
    year = cogann.split('/')[-2].split('-')[0]
    if year != '2019':
        continue
    gunann = cogann.replace("_cogito", "")
    if not os.path.isfile(gunann):
#         print('missing')
        continue
    cog = json.load(open(cogann))
    gun = json.load(open(gunann))# ['detections']
    if gun['detections'] is None:
        continue
    if cog is None:
        continue
    if len(cog) == 0:
        continue
#     if len(cog) == len(gun):
#         print(cog)
#         print(gun)
    # cogito has annotated stuff but gunnar has deleted everything
    if len(gun['detections']) > 0:
        diff = find_difference(cog, gun['detections'])
        bad.append((cogann, diff))
#     if len(cog) > len(gun):
#         for cogbox in cog:
#             print(cogbox)
    

In [ ]:
len(bad)

In [ ]:
np.sum([len(t[1]) for t in bad])

In [ ]:
for b in tqdm(bad):
    image_path = b[0].replace('_cogito.json', '.jpg')
    name = os.path.basename(image_path)
    image = cv2.imread(image_path)
    for (i,lice) in enumerate(b[1]):
        position = lice['position']
        x1 = position['top']
        y1 = position['left']
        width = position['width']
        height=position['height']
        tosave = image[x1:x1+height, y1:y1+width, :]
        destination = os.path.join('/root/data/lice-data/declassified/{}'.format(name.replace('.jpg', '_{}.jpg'.format(i))))
        cv2.imwrite(destination, tosave)

# OLD CODE

In [ ]:
annotation = glob.glob("/root/data/lice-data/crops/blom-kjeppevikholmen/2/*/*.json")

In [ ]:
input_shape = (64, 64)
nimage = 2000

In [ ]:
random_annotations = np.random.choice(annotation, nimage, replace=False)
c = 9900
for path in tqdm(random_annotations):
    ann = json.load(open(path))
    detections = ann['detections']
    image_path = path.replace('json', 'jpg')
    image = cv2.imread(image_path)
    
#     f, ax = plt.subplots(1, figsize=(20, 10))
#     ax.imshow(image)
    
    if not detections:
        continue
    proposal = []
    for lice in detections:
        location = lice['location']
        position = lice['position']
        x1 = position['top']
        y1 = position['left']
        width = position['width']
        height = position['height']
        
#         rect = Rectangle((y1, x1) ,width, height,
#                  linewidth=2, edgecolor='r', facecolor='none')
#         ax.add_patch(rect) 
        
        for _ in range(2):
            delta_x = np.random.randint(0, 100)
            delta_y = np.random.randint(0, 100)

            if location == 'TOP':
                new_x = x1 + height + delta_x
                coin = np.random.rand()
                if coin > 0.5:
                    new_y = y1 + width + delta_y
                else:
                    new_y = y1 - input_shape[1] - delta_y

            elif location == 'BOTTOM':
                new_x = x1 - input_shape[0] - delta_x
                coin = np.random.rand()
                if coin > 0.5:
                    new_y = y1 + width + delta_y
                else:
                    new_y = y1 - input_shape[1] - delta_y
            
            else:
                continue
                    
#             rect = Rectangle((new_y, new_x) , 64, 64,
#                      linewidth=2, edgecolor='blue', facecolor='none')
#             ax.add_patch(rect)
            
            random_patch = image[new_x:new_x+input_shape[0], new_y:new_y+input_shape[1], :]
            cv2.imwrite('/root/data/lice-data/random_patch/random_patch_{}.jpg'.format(c),
                        random_patch)
            c += 1
            x1 = new_x
            y1 = new_y
            width = 64
            height = 64
#     plt.show()
        


### VIZ PATCHES

In [ ]:
import glob
import json
import numpy as np
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os
from albumentations import PadIfNeeded, HorizontalFlip, VerticalFlip, Compose, RandomRotate90

In [ ]:
patches = glob.glob('/root/data/lice-data/declassified//*')

In [ ]:
input_shape = [64, 64, 3]

In [ ]:
def load_lice(img_path):
    '''load lice'''
    aug = Compose([PadIfNeeded(p=1.0, min_height=input_shape[0], min_width=input_shape[1], border_mode=0),
               HorizontalFlip(p=0.5),
               RandomRotate90(p=0.5)])
    image = cv2.imread(img_path)
    height, width, _ = image.shape
    if height > width:
        ratio = input_shape[0] / float(height)
        x = cv2.resize(image, (int(width*ratio), input_shape[1]), interpolation=cv2.INTER_LINEAR)
    else:
        ratio = input_shape[1] / float(width)
        x = cv2.resize(image, (input_shape[0], int(height*ratio)), interpolation=cv2.INTER_LINEAR)
    augmented = aug(image=x)
    x = augmented["image"]
    return x 

In [ ]:
print(len(patches))
for path in tqdm(patches):
    patch = load_lice(path)
    if patch.shape[0] != 64 or patch.shape[1] != 64:
        print(patch.shape)
        print(path)

In [ ]:
# c = 0
# print(len(patches))
# for path in tqdm(patches):
#     patch = cv2.imread(path)
#     if patch.shape[0] != 64 or patch.shape[1] != 64:
#         print(patch.shape)
#         print(path)
#         os.remove(path)
#     if patch is None:
#         os.remove(path)
#         c+=1

In [ ]:
for _ in range(100):
    path = np.random.choice(patches)
    patch = cv2.imread(path)
    print(path)
    plt.imshow(patch)
    plt.show()

In [ ]:
from datetime import datetime

In [ ]:
datetime.utcfromtimestamp(1543326617448/1000.0)

In [ ]:
path = '/root/data/lice-data/crops/blom-kjeppevikholmen/2/2018-11-27/left_blom-kjeppevikholmen_2_1543326617448_1868_880_4096_2903.jpg'

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(cv2.imread(path)[:700, :800])

In [ ]:
left_blom-kjeppevikholmen_2_1543326617448_1868_880_4096_2903